# Projet de DataMining :  

Dans ce projet nous allons recommander des images en fonctions d'images déjà "liké".
Pour cela nous avons commencé par traiter une base de 733 images en recupérant le maximun de données. On recupére notament les données Exif, la taille de l'image, son orientation, ect ...  
La base de données d'images vient du lien suivant:
https://www.kaggle.com/aksha05/flower-image-dataset
## Requirements : 

In [ ]:
from PIL import Image
import json

## Récupération des données:

In [23]:
img = Image.open('images/flowers/bougainvillea_00003.jpg')

tupleSize = img.size
if tupleSize[0] >= tupleSize[1] : 
    



IndentationError: expected an indented block (Temp/ipykernel_17240/3107516493.py, line 5)